In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from climate import EarthModel, fit_eruption_data
from numpy.random import exponential

In [ ]:
def seconds(years):
    return years * 365.25 * 24 * 3600

In [ ]:
# Simulate eruptions
np.random.seed(42)
test_repose_times = np.array(
    [100, 50, 20, 20, 50, 100]
) * 365.25 * 24 * 3600
zone_eruption_times = np.array([
    exponential(test_repose_times[i], 200).cumsum() for i in range(6)
])
eruption_zones = []
eruption_times = []
for i in range(6):
    for t in zone_eruption_times[i]:
        eruption_zones.append(i)
        eruption_times.append(t)

In [ ]:
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
model.build_eruptions(eruption_zones, eruption_times)
model.t0 = 0
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'
model.build()
model.solve()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for i in range(model.size):
    ax.plot(
        model.t, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i), zorder=1)

ax.legend()
ax.set_xlim((0, seconds(100)))
# Plot zone 0 eruption times as vertical lines
for i in range(6):
    for t in zone_eruption_times[i]:
        ax.axvline(
            t, color='C{}'.format(i),
            zorder=0, linewidth=0.7, linestyle='dashed'
        )

In [ ]:
# Try albedo effects
model = EarthModel('parameters.json')
model.set_occlusion(
    fit_eruption_data(['eruption_1.csv', 'eruption_2.csv'])
)
# model.build_eruptions(eruption_zones, eruption_times)
model.build_eruptions([], [])
model.t0 = 0
model.T0 = [274.12, 279.34, 282.26, 280.88, 279.71, 274.93]
model.tf = 100 * 3.154e+7 # Years
model.tn = 500
model.method = 'vode'
model.build_ode(
    lambda t, T: model.flux_balance_albedo(t, T)
)
model.solve()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
for i in range(model.size):
    ax.plot(
        model.t, model.T[:, i],
        linewidth=2, label='Zone {}'.format(i), zorder=1)

ax.legend()
ax.set_xlim((0, seconds(100)))
# Plot zone 0 eruption times as vertical lines
for i in range(6):
    for t in zone_eruption_times[i]:
        ax.axvline(
            t, color='C{}'.format(i),
            zorder=0, linewidth=0.7, linestyle='dashed'
        )